# WhatsApp Reply Agent

An intelligent agent that can automatically reply to WhatsApp messages with context-aware responses and scheduling capabilities.

## Features:
- **Smart Reply Generation**: Analyzes incoming messages and generates appropriate responses
- **Message Scheduling**: Schedule messages to be sent at specific times
- **Contact Management**: Handle different contacts with personalized responses
- **Message Analysis**: Understand message tone, urgency, and context

In [12]:
import datetime
import threading
import time
import json
from typing import Dict, List, Optional
from dataclasses import dataclass
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

In [13]:
client = OpenAI()

In [14]:
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

In [15]:
print("                                                          Name : Yuvaraj S\n                                                    Register number : 212222240119")

                                                          Name : Yuvaraj S
                                                    Register number : 212222240119


In [16]:
chat_completion.choices[0].message.content


'Hello! How can I assist you today?'

In [17]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

In [18]:
@dataclass
class Contact:
    name: str
    number: str
    relationship: str  # friend, family, business, etc.
    personality: str   # casual, formal, humorous, etc.

@dataclass
class ScheduledMessage:
    contact: str
    message: str
    send_time: datetime.datetime
    sent: bool = False

whatsapp_prompt = """
You are a WhatsApp Reply Agent that helps craft appropriate responses and manage messaging.
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.

Use Thought to analyze the incoming message and determine the best response strategy.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

analyze_message:
e.g. analyze_message: Hey! How are you doing today?
Analyzes the tone, urgency, and type of incoming message

generate_reply:
e.g. generate_reply: casual greeting from friend
Generates an appropriate reply based on message analysis and contact relationship

schedule_message:
e.g. schedule_message: +1234567890, Happy Birthday!, 2024-08-02 09:00
Schedules a message to be sent at a specific time (format: number, message, YYYY-MM-DD HH:MM)

get_contact_info:
e.g. get_contact_info: +1234567890
Retrieves information about a contact including relationship and preferred communication style

send_immediate_reply:
e.g. send_immediate_reply: +1234567890, Thanks for the message!
Sends an immediate reply to the specified number

Example session:

Question: I received "Hey! Are we still on for dinner tonight?" from +1234567890
Thought: I need to analyze this message and check who this contact is, then generate an appropriate response
Action: get_contact_info: +1234567890
PAUSE

You will be called again with this:

Observation: Contact: John Smith, Relationship: friend, Style: casual

Then continue analyzing and responding...

Answer: Generated appropriate reply and scheduled/sent message as requested.
""".strip()

In [19]:
contacts_db = {
    "+918766766556": Contact("John Smith", "+918766766556", "friend", "casual"),
}

scheduled_messages: List[ScheduledMessage] = []
sent_messages: List[Dict] = []

def analyze_message(message_text):
    message_lower = message_text.lower()

    urgent_keywords = ['urgent', 'asap', 'emergency', 'now', 'immediately', 'help']
    urgency = "high" if any(keyword in message_lower for keyword in urgent_keywords) else "normal"

    if any(word in message_lower for word in ['hey', 'hi', 'hello', '😊', '👋']):
        tone = "friendly"
    elif any(word in message_lower for word in ['thanks', 'thank you', 'appreciate']):
        tone = "grateful"
    elif '?' in message_text:
        tone = "questioning"
    elif '!' in message_text:
        tone = "excited"
    else:
        tone = "neutral"

    if '?' in message_text:
        msg_type = "question"
    elif any(word in message_lower for word in ['meeting', 'dinner', 'lunch', 'appointment']):
        msg_type = "scheduling"
    elif any(word in message_lower for word in ['birthday', 'congratulations', 'celebration']):
        msg_type = "celebration"
    else:
        msg_type = "general"
    
    return f"Analysis: Urgency={urgency}, Tone={tone}, Type={msg_type}, Length={len(message_text.split())} words"

def generate_reply(context_and_style):
    context = context_and_style.lower()

    if "business" in context or "formal" in context:
        if "question" in context:
            return "Thank you for your message. I'll review this and get back to you shortly."
        elif "meeting" in context:
            return "I'll check my calendar and confirm the meeting details with you."
        else:
            return "Thank you for reaching out. I appreciate your message."
    
    elif "friend" in context or "casual" in context:
        if "question" in context:
            return "Hey! Let me check on that and get back to you "
        elif "dinner" in context or "lunch" in context:
            return "Absolutely! Looking forward to it! 🍽️"
        elif "greeting" in context:
            return "Hey there! I'm doing great, thanks for asking! How about you?"
        else:
            return "Thanks for the message! Hope you're doing well "
    
    elif "family" in context:
        if "celebration" in context:
            return "Thank you so much! Love you too! "
        elif "question" in context:
            return "Of course! Let me check and let you know soon."
        else:
            return "Thanks for checking in! Love you! "
    
    else:
        return "Thank you for your message. I'll respond appropriately soon."

def schedule_message(message_details):
    try:
        parts = message_details.split(', ', 2)
        if len(parts) != 3:
            return "Error: Format should be 'number, message, YYYY-MM-DD HH:MM'"
        
        number, message, time_str = parts
        send_time = datetime.datetime.strptime(time_str.strip(), "%Y-%m-%d %H:%M")
        
        scheduled_msg = ScheduledMessage(
            contact=number.strip(),
            message=message.strip(),
            send_time=send_time
        )
        scheduled_messages.append(scheduled_msg)
        
        return f"Message scheduled for {send_time.strftime('%Y-%m-%d %H:%M')} to {number}"
    except Exception as e:
        return f"Error scheduling message: {e}"

def get_contact_info(phone_number):
    phone_number = phone_number.strip()
    if phone_number in contacts_db:
        contact = contacts_db[phone_number]
        return f"Contact: {contact.name}, Relationship: {contact.relationship}, Style: {contact.personality}"
    else:
        return f"Unknown contact: {phone_number}. Using default: neutral relationship, professional style"

def send_immediate_reply(message_details):
    try:
        parts = message_details.split(', ', 1)
        if len(parts) != 2:
            return "Error: Format should be 'number, message'"
        
        number, message = parts

        sent_msg = {
            "number": number.strip(),
            "message": message.strip(),
            "timestamp": datetime.datetime.now().isoformat(),
            "status": "sent"
        }
        sent_messages.append(sent_msg)
        
        return f"Message sent to {number}: '{message}'"
    except Exception as e:
        return f"Error sending message: {e}"

whatsapp_actions = {
    "analyze_message": analyze_message,
    "generate_reply": generate_reply,
    "schedule_message": schedule_message,
    "get_contact_info": get_contact_info,
    "send_immediate_reply": send_immediate_reply
}

In [20]:
def whatsapp_query(incoming_message, sender_number, auto_reply=True, schedule_time=None):
    print("📱 WhatsApp Reply Agent Activated")
    print("=" * 50)
    print(f"📩 Incoming Message: '{incoming_message}'")
    print(f"📞 From: {sender_number}")
    
    if schedule_time:
        print(f"⏰ Scheduled for: {schedule_time}")

    if schedule_time:
        query_text = f"I received '{incoming_message}' from {sender_number}. Please generate an appropriate reply and schedule it for {schedule_time}."
    else:
        query_text = f"I received '{incoming_message}' from {sender_number}. Please analyze it and generate an appropriate reply{' and send it immediately' if auto_reply else ''}."

    i = 0
    max_turns = 6
    whatsapp_bot = Agent(whatsapp_prompt)
    next_prompt = query_text
    
    while i < max_turns:
        i += 1
        print(f"\n🤖 Agent Turn {i}:")
        result = whatsapp_bot(next_prompt)
        print(result)

        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        
        if actions:
            action, action_input = actions[0].groups()
            if action not in whatsapp_actions:
                print(f" Unknown action: {action}: {action_input}")
                break
                
            print(f"\n Executing: {action}({action_input})")
            observation = whatsapp_actions[action](action_input)
            print(f"📋 Result: {observation}")
            next_prompt = "Observation: {}".format(observation)
        else:
            print("\n WhatsApp agent task completed!")
            return result
            
    print(f"\n⏰ Agent completed after {i} turns")
    return result

def check_scheduled_messages():
    print("\n Scheduled Messages:")
    print("-" * 40)
    
    if not scheduled_messages:
        print("No scheduled messages.")
        return
    
    current_time = datetime.datetime.now()
    
    for i, msg in enumerate(scheduled_messages, 1):
        status = " READY TO SEND" if msg.send_time <= current_time and not msg.sent else "⏳ SCHEDULED"
        print(f"{i}. To: {msg.contact}")
        print(f"   Message: {msg.message}")
        print(f"   Send Time: {msg.send_time.strftime('%Y-%m-%d %H:%M')}")
        print(f"   Status: {status}")
        print()

def show_sent_messages():
    print("\n Sent Messages History:")
    print("-" * 40)
    
    if not sent_messages:
        print("No messages sent yet.")
        return
    
    for i, msg in enumerate(sent_messages, 1):
        timestamp = datetime.datetime.fromisoformat(msg['timestamp'])
        print(f"{i}. To: {msg['number']}")
        print(f"   Message: {msg['message']}")
        print(f"   Sent: {timestamp.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"   Status: {msg['status']}")
        print()

## Test WhatsApp Reply Agent

Let's test the WhatsApp agent with different scenarios:

In [21]:
action_re = re.compile(r"^Action:\s*(\w+):\s*(.+)$")
print("🧪 TEST 1: Casual Friend Message")
whatsapp_query(
    incoming_message="Hey! Are we still on for dinner tonight? 😊",
    sender_number="+918766766556",
    auto_reply=True
)

🧪 TEST 1: Casual Friend Message
📱 WhatsApp Reply Agent Activated
📩 Incoming Message: 'Hey! Are we still on for dinner tonight? 😊'
📞 From: +918766766556

🤖 Agent Turn 1:
Action: get_contact_info: +918766766556
PAUSE

 Executing: get_contact_info(+918766766556)
📋 Result: Contact: John Smith, Relationship: friend, Style: casual

🤖 Agent Turn 2:
Action: analyze_message: Hey! Are we still on for dinner tonight? 😊
PAUSE

 Executing: analyze_message(Hey! Are we still on for dinner tonight? 😊)
📋 Result: Analysis: Urgency=normal, Tone=friendly, Type=question, Length=9 words

🤖 Agent Turn 3:
Action: generate_reply: casual confirmation for dinner plans with friend
PAUSE

 Executing: generate_reply(casual confirmation for dinner plans with friend)
📋 Result: Absolutely! Looking forward to it! 🍽️

🤖 Agent Turn 4:
Action: send_immediate_reply: +918766766556, Absolutely! Looking forward to it! 🍽️
PAUSE

 Executing: send_immediate_reply(+918766766556, Absolutely! Looking forward to it! 🍽️)
📋 Result: Me

'Answer: Generated an appropriate reply and sent the message immediately.'

In [22]:
check_scheduled_messages()

show_sent_messages()


 Scheduled Messages:
----------------------------------------
No scheduled messages.

 Sent Messages History:
----------------------------------------
1. To: +918766766556
   Message: Absolutely! Looking forward to it! 🍽️
   Sent: 2025-08-01 05:47:36
   Status: sent



## Real-World Implementation Setup

To use this WhatsApp agent in production, follow these steps:

### 1. Install Required Packages
```bash
pip install pywhatkit
pip install opencv-python  
pip install schedule  
```

### 2. Setup WhatsApp Web
- Install PyWhatKit: `pip install pywhatkit`
- First run will open WhatsApp Web - scan QR code
- Agent will control your WhatsApp through the web interface

### 3. Replace Simulation Functions
Update the `send_immediate_reply` function:
```python
import pywhatkit as pwk

def send_immediate_reply(message_details):
    parts = message_details.split(', ', 1)
    number, message = parts
    pwk.sendwhatmsg_instantly(number, message)
    return f"Message sent to {number}: '{message}'"
```

### 4. Advanced Features You Can Add:
- **Contact Detection**: Auto-detect contact names from phone number
- **Sentiment Analysis**: Respond based on emotional tone
- **Voice Messages**: Convert text to voice messages
- **Image/Media Responses**: Send appropriate media based on context

### 5. Security Considerations:
- Store contact data securely
- Implement rate limiting
- Add authentication for sensitive operations
- Log all automated actions
- Include manual override options

### 6. Usage Examples:
```python
# Simple auto-reply
whatsapp_query("Thanks for the info!", "+1234567890")

# Scheduled birthday message
whatsapp_query("", "+1234567890", schedule_time="2024-08-15 09:00")

# Business reply with delay
whatsapp_query("Meeting request received", "+1122334455", schedule_time="2024-08-02 08:30")
```